<a href="https://colab.research.google.com/github/Jarvis-Geun/DeepLearning-Wiki/blob/main/Semantic-Segmentation/U-Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Reference
- [Semantic Segmentation in Self-driving Cars](https://blog.jovian.ai/semantic-segmentation-in-self-driving-cars-3cb89aa08e9b)

## Step 1. 모델 설계

### 1.1 라이브러리 불러오기

In [ ]:
import os
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from tqdm.notebook import tqdm

모델 설계하는데 필요한 라이브러리를 불러옵니다.
[PIL 라이브러리](https://ko.wikipedia.org/wiki/Python_Imaging_Library)

### 1.2 GPU 설정하기

In [ ]:
# GPU 사용이 가능할 경우, GPU를 사용할 수 있게 함.
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device = torch.device(device)
print(device)

cuda:0


### 1.3 파일 시스템

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# root_path = '/content/drive/MyDrive/start-knitting-girls/UNet/dataset/dataset/' # 입력 이미지 폴더, /content/ 추가

# data_dir = root_path

# # data_dir의 경로(문자열)와 train(문자열)을 결합해서 train_dir(train 폴더의 경로)에 저장합니다.
# train_dir = os.path.join(data_dir, "train/images")

# # data_dir의 경로(문자열)와 val(문자열)을 결합해서 val_dir(val 폴더의 경로)에 저장합니다.
# val_dir = os.path.join(data_dir, "val/images")

# # train_dir 경로에 있는 모든 파일을 리스트의 형태로 불러와서 train_fns에 저장합니다.
# train_fns = os.listdir(train_dir)

# # val_dir 경로에 있는 모든 파일을 리스트의 형태로 불러와서 val_fns에 저장합니다.
# val_fns = os.listdir(val_dir)

In [ ]:
# 각각의 데이터 경로 설정
# data_dir = ''
train_dir = ''
val_dir = ''

train_fns = os.listdir(train_dir)

val_fns = os.listdir(val_dir)

In [ ]:
# root_path = '/content/drive/MyDrive/start-knitting-girls/UNet/dataset/sweater/' # 입력 이미지 폴더, /content/ 추가

# data_dir = root_path

# # data_dir의 경로(문자열)와 train(문자열)을 결합해서 train_dir(train 폴더의 경로)에 저장합니다.
# train_dir = os.path.join(data_dir, "train")

# # data_dir의 경로(문자열)와 val(문자열)을 결합해서 val_dir(val 폴더의 경로)에 저장합니다.
# val_dir = os.path.join(data_dir, "val")

# # train_dir 경로에 있는 모든 파일을 리스트의 형태로 불러와서 train_fns에 저장합니다.
# train_fns = os.listdir(train_dir)

# # val_dir 경로에 있는 모든 파일을 리스트의 형태로 불러와서 val_fns에 저장합니다.
# val_fns = os.listdir(val_dir)

train_fns의 길이는 2975이며 val_fns의 길이는 500입니다. 이는 데이터셋인 **Cityscape Dataset**의 학습(train) 및 검증(validation) 데이터와 일치하는 것을 확인할 수 있습니다.

### 1.4 샘플 이미지 검색
경로를 지정했으므로 이제 이 경로를 사용하여 샘플 이미지를 불러오도록 하겠습니다. 이 과정은 생략해도 되지만 불러오는 과정이 원활하게 동작하는지 확인하기 위해 실습해보도록 하겠습니다.

In [ ]:
# train_dir(문자열)와 train_fns[0](문자열)의 경로를 결합하여 sample_image_fp(샘플 이미지의 경로)에 저장합니다.
sample_image_fp = os.path.join(train_dir, train_fns[0])

# PIL 라이브러리의 Image 모듈을 사용하여, sample_image_fp를 불러옵니다.
# RGB 형태로 변환하여 sample_image에 저장하는 것으로 이해했는데, ".convert("RGB")" 코드는 없어도 될 것 같습니다.
# Image.open() 함수 자체가 RGB의 형태로 불러오는 것으로 이해했습니다. 확실하지 않습니다...
sample_image = Image.open(sample_image_fp).convert("RGB")

plt.imshow(sample_image)
plt.show()

[plt.show() vs plt.imshow()](https://stackoverflow.com/questions/3497578/matplotlib-plot-and-imshow)

### 1.5 Output Label 정의하기

In [ ]:
num_items = 1000

# 0~255 사이의 숫자를 3*num_items번 랜덤하게 뽑기
color_array = np.random.choice(range(256), 3*num_items).reshape(-1, 3)
print(color_array.shape)

In [ ]:
num_classes = 10

# K-means clustering 알고리즘을 사용하여 label_model에 저장합니다.
label_model = KMeans(n_clusters = num_classes)
label_model.fit(color_array)

[K-means clustering](https://blog.mathpresso.com/mathpresso-%EB%A8%B8%EC%8B%A0-%EB%9F%AC%EB%8B%9D-%EC%8A%A4%ED%84%B0%EB%94%94-9-%EA%B5%B0%EC%A7%91%ED%99%94-clustering-542390bb4b74)

[label_model.fit(color_array)](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#:~:text=fit(X%2C%20y%3DNone%2C%20sample_weight%3DNone))

In [ ]:
#  # 이전에 샘플이미지에서 볼 수 있듯이, original image와 labeled image가 연결되어 있는데 이를 분리해줍니다.
#  def split_image(image) :
#     image = np.array(image)

#     # 이미지의 크기가 256 x 512 였는데 이를 original image와 labeled image로 분리하기 위해 리스트로 슬라이싱 합니다.
#     # 그리고 분리된 이미지를 각각 cityscape(= original image)와 label(= labeled image)에 저장합니다.
#     cityscape, label = image[:, :256, :], image[:, 256:, :]
#     return cityscape, label


# # # 바로 이전 코드에서 정의한 split_image() 함수를 이용하여 sample_image를 분리한 후, cityscape과 label에 각각 저장합니다.
# cityscape, label = split_image(sample_image)

# label_class = label_model.predict(label.reshape(-1, 3)).reshape(256, 256)
# fig, axes = plt.subplots(1, 3, figsize = (15, 5))
# axes[0].imshow(cityscape)
# axes[1].imshow(label)
# axes[2].imshow(label_class)

# plt.show()

In [ ]:
def split_image(image):
    # 이미지 크기 확인
    image = np.array(image)
    height, width, _ = image.shape
    cityscape_width = width // 2  # 이미지의 절반 너비를 계산
    cityscape, label = image[:, :cityscape_width, :], image[:, cityscape_width:, :]
    return cityscape, label

# 이미지 분리
cityscape, label = split_image(sample_image)

# 레이블 클래스 예측
label_height, label_width = label.shape[:2]
label_class = label_model.predict(label.reshape(-1, 3)).reshape(label_height, label_width)

# 시각화
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(cityscape)
axes[1].imshow(label)
axes[2].imshow(label_class, cmap='viridis')  # 컬러맵 지정
plt.show()


### 1.6 데이터셋 정의하기

In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image

class CityscapeDataset(Dataset):

    def __init__(self, image_dir, label_model, target_size=(256, 256)):
        self.image_dir = image_dir
        self.image_fns = os.listdir(image_dir)
        self.label_model = label_model
        self.target_size = target_size  # 원하는 이미지 크기 (가로, 세로)

    def __len__(self):
        return len(self.image_fns)

    def __getitem__(self, index):
        image_fn = self.image_fns[index]
        image_fp = os.path.join(self.image_dir, image_fn)

        # 이미지 로드
        image = Image.open(image_fp).convert("RGB")
        image = np.array(image)

        # 이미지 분리
        cityscape, label = self.split_image(image)

        # 크기 조정
        cityscape = self.resize_image(cityscape)
        label = self.resize_image(label)

        # 레이블 클래스 변환
        label_class = self.label_model.predict(label.reshape(-1, 3)).reshape(self.target_size[1], self.target_size[0])
        label_class = torch.Tensor(label_class).long()

        # 시티스케이프 이미지 변환
        cityscape = self.transform(cityscape)

        return cityscape, label_class

    def split_image(self, image):
        """
        이미지를 두 부분으로 나눔: 시티스케이프(좌측)와 레이블(우측).
        """
        cityscape = image[:, :256, :]  # 좌측 256 픽셀
        label = image[:, 256:, :]      # 우측 나머지
        return cityscape, label

    def resize_image(self, image):
        """
        이미지를 target_size로 리사이즈.
        """
        image = Image.fromarray(image)
        image = image.resize(self.target_size, Image.BILINEAR)  # Bilinear로 리사이즈
        return np.array(image)

    def transform(self, image):
        """
        이미지를 PyTorch 텐서로 변환 및 정규화.
        """
        transform_ops = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.56, 0.406), std=(0.229, 0.224, 0.225))
        ])
        return transform_ops(image)


In [ ]:
# dataset = CityscapeDataset(train_dir, label_model)
# print(len(dataset))

# cityscape, label_class = dataset[0]
# print(cityscape.shape)
# print(label_class.shape)

# 몇개의 데이터셋, 라벨 클래스의 모양 출력

In [ ]:
# # Value error

# class CityscapeDataset(Dataset):
#     def __init__(self, image_dir, label_model):
#         self.image_dir = image_dir
#         self.image_fns = os.listdir(image_dir)
#         self.label_model = label_model

#     def __len__(self):
#         return len(self.image_fns)

#     def __getitem__(self, index): # value error 수정
#         image_fn = self.image_fns[index]
#         image_fp = os.path.join(self.image_dir, image_fn)
#         image = Image.open(image_fp)
#         image = np.array(image)
#         cityscape, label = self.split_image(image)

#         # Get the actual height and width of the label image
#         label_height = label.shape[0]
#         label_width = label.shape[1]

#         # Reshape based on actual dimensions
#         label_class = self.label_model.predict(label.reshape(-1, 3)).reshape(label_height, label_width)
#         label_class = torch.Tensor(label_class).long()
#         cityscape = self.transform(cityscape)
#         return cityscape, label_class


#     def split_image(self, image):
#         image = np.array(image)
#         # cityscape, label = image[:, :256, :], image[:, 256:, :]
#         split_point = image.shape[1] // 2 # 영향을 주지 않음
#         cityscape, label = image[:, :split_point, :], image[:, split_point:, :]
#         return cityscape, label

#     def transform(self, image):
#         transform_ops = transforms.Compose([
#             transforms.ToTensor(),
#             transforms.Normalize(mean=(0.485, 0.56, 0.406), std=(0.229, 0.224, 0.225))
#         ])
#         return transform_ops(image)

# print(cityscape.shape, label_class.shape)
# print(len(cityscape), len(label_class))

위의 출력 결과를 통해, 학습 데이터가 2975개 있다는 것을 다시 한번 확인할 수 있습니다. 또한, cityscape과 label_class의 shape도 알 수 있습니다. cityscape의 경우 **transforms.ToTensor()** 를 통과하여 [3, 256, 256]의 텐서 형태를 가지게 되는 것을 확인할 수 있습니다.
<br/>

### 1.7 U-Net 모델 정의하기
[이전 포스팅](https://velog.io/@jarvis_geun/U-Net-%ED%86%BA%EC%95%84%EB%B3%B4%EA%B8%B0in-depth)에서 다룬 U-Net 모델을 사용하여 **Sementic Segmentation**을 진행하겠습니다. 아래의 사진을 바탕으로 U-Net 모델을 만듭니다.  
  
[![](https://images.velog.io/images/jarvis_geun/post/3e74c2f1-a1e9-4248-b9fc-680fce3a0d8c/image.png)](https://velog.io/@jarvis_geun/U-Net-%ED%86%BA%EC%95%84%EB%B3%B4%EA%B8%B0in-depth)

In [ ]:
class UNet(nn.Module):

    def __init__(self, num_classes):
        super(UNet, self).__init__()
        self.num_classes = num_classes
        self.contracting_11 = self.conv_block(in_channels=3, out_channels=64)
        self.contracting_12 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.contracting_21 = self.conv_block(in_channels=64, out_channels=128)
        self.contracting_22 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.contracting_31 = self.conv_block(in_channels=128, out_channels=256)
        self.contracting_32 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.contracting_41 = self.conv_block(in_channels=256, out_channels=512)
        self.contracting_42 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.middle = self.conv_block(in_channels=512, out_channels=1024)
        self.expansive_11 = nn.ConvTranspose2d(in_channels=1024, out_channels=512, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.expansive_12 = self.conv_block(in_channels=1024, out_channels=512)
        self.expansive_21 = nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.expansive_22 = self.conv_block(in_channels=512, out_channels=256)
        self.expansive_31 = nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.expansive_32 = self.conv_block(in_channels=256, out_channels=128)
        self.expansive_41 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.expansive_42 = self.conv_block(in_channels=128, out_channels=64)
        self.output = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
	# 1x1 convolution layer 추가
        self.output1 = nn.Conv2d(in_channels=64, out_channels=num_classes, kernel_size=1, stride=1, padding=1)

    def conv_block(self, in_channels, out_channels):
        block = nn.Sequential(nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(num_features=out_channels),
                                    nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(num_features=out_channels))
        return block

    def forward(self, X):
        contracting_11_out = self.contracting_11(X) # [-1, 64, 256, 256]
        contracting_12_out = self.contracting_12(contracting_11_out) # [-1, 64, 128, 128]
        contracting_21_out = self.contracting_21(contracting_12_out) # [-1, 128, 128, 128]
        contracting_22_out = self.contracting_22(contracting_21_out) # [-1, 128, 64, 64]
        contracting_31_out = self.contracting_31(contracting_22_out) # [-1, 256, 64, 64]
        contracting_32_out = self.contracting_32(contracting_31_out) # [-1, 256, 32, 32]
        contracting_41_out = self.contracting_41(contracting_32_out) # [-1, 512, 32, 32]
        contracting_42_out = self.contracting_42(contracting_41_out) # [-1, 512, 16, 16]
        middle_out = self.middle(contracting_42_out) # [-1, 1024, 16, 16]
        expansive_11_out = self.expansive_11(middle_out) # [-1, 512, 32, 32]
        expansive_12_out = self.expansive_12(torch.cat((expansive_11_out, contracting_41_out), dim=1)) # [-1, 1024, 32, 32] -> [-1, 512, 32, 32]
        expansive_21_out = self.expansive_21(expansive_12_out) # [-1, 256, 64, 64]
        expansive_22_out = self.expansive_22(torch.cat((expansive_21_out, contracting_31_out), dim=1)) # [-1, 512, 64, 64] -> [-1, 256, 64, 64]
        expansive_31_out = self.expansive_31(expansive_22_out) # [-1, 128, 128, 128]
        expansive_32_out = self.expansive_32(torch.cat((expansive_31_out, contracting_21_out), dim=1)) # [-1, 256, 128, 128] -> [-1, 128, 128, 128]
        expansive_41_out = self.expansive_41(expansive_32_out) # [-1, 64, 256, 256]
        expansive_42_out = self.expansive_42(torch.cat((expansive_41_out, contracting_11_out), dim=1)) # [-1, 128, 256, 256] -> [-1, 64, 256, 256]
        output_out = self.output(expansive_42_out) # [-1, 64, 256, 256] -> [-1, 64, 256, 256]
        output_out1 = self.output(output_out) # [-1, num_classes, 256, 256]

        return output_out1

In [ ]:
# AttributeError: '_SingleProcessDataLoaderIter' object has no attribute 'next'
from torch.utils.data import DataLoader

model = UNet(num_classes=num_classes)

dataset = CityscapeDataset(train_dir, label_model)

data_loader = DataLoader(dataset, batch_size = 4)


# X, Y = iter(data_loader).next()
X, Y = next(iter(data_loader)) # value error
print(X.shape, Y.shape)

In [ ]:
Y_pred = model(X)
print(Y_pred.shape)

## Step 2. 모델 학습

In [ ]:
batch_size = 16

epochs = 50
lr = 0.01

dataset = CityscapeDataset(train_dir, label_model)
data_loader = DataLoader(dataset, batch_size = batch_size)

model = UNet(num_classes = num_classes).to(device)

# 손실함수 정의
criterion = nn.CrossEntropyLoss()
# Optimizer 정의
optimizer = optim.Adam(model.parameters(), lr = lr)


step_losses = []
epoch_losses = []

for epoch in tqdm(range(epochs)) :
  epoch_loss = 0

  for X, Y in tqdm(data_loader, total = len(data_loader), leave = False) :
    X, Y = X.to(device), Y.to(device)
    optimizer.zero_grad()
    Y_pred = model(X)
    loss = criterion(Y_pred, Y)
    loss.backward()
    optimizer.step()
    epoch_loss += loss.item()
    step_losses.append(loss.item())
  epoch_losses.append(epoch_loss/len(data_loader))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].plot(step_losses)
axes[1].plot(epoch_losses)

plt.show()

## Step 3. 모델 평가하기

In [ ]:
model_name = "UNet.pth"
torch.save(model.state_dict(), root_path + model_name)

In [ ]:
model_path = root_path + model_name
model_ = UNet(num_classes = num_classes).to(device)
model_.load_state_dict(torch.load(model_path))

In [ ]:
test_batch_size = 8
dataset = CityscapeDataset(val_dir, label_model)
data_loader = DataLoader(dataset, batch_size = test_batch_size)

X,Y = next(iter(data_loader))
X,Y = X.to(device), Y.to(device)
Y_pred = model_(X)
print(Y_pred.shape)
Y_pred = torch.argmax(Y_pred, dim=1)
print(Y_pred.shape)

In [ ]:
inverse_transform = transforms.Compose([
    transforms.Normalize((-0.485/0.229, -0.456/0.224, -0.406/0.225), (1/0.229, 1/0.224, 1/0.225))
])

In [ ]:
fig, axes = plt.subplots(test_batch_size, 3, figsize=(3*5, test_batch_size*5))

iou_scores = []

for i in range(test_batch_size):

    landscape = inverse_transform(X[i]).permute(1, 2, 0).cpu().detach().numpy()
    label_class = Y[i].cpu().detach().numpy()
    label_class_predicted = Y_pred[i].cpu().detach().numpy()

    # IOU score
    intersection = np.logical_and(label_class, label_class_predicted)
    union = np.logical_or(label_class, label_class_predicted)
    iou_score = np.sum(intersection) / np.sum(union)
    iou_scores.append(iou_score)

    axes[i, 0].imshow(landscape)
    axes[i, 0].set_title("Landscape")
    axes[i, 1].imshow(label_class)
    axes[i, 1].set_title("Label Class")
    axes[i, 2].imshow(label_class_predicted)
    axes[i, 2].set_title("Label Class - Predicted")

plt.show()

## Step 4. IOU Score

In [ ]:
print(sum(iou_scores) / len(iou_scores))